<a href="https://colab.research.google.com/github/syedhammadahmed/quantumcomputings20/blob/master/hw_2/hw_2_problem_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## HW 1 ##

### Problem 2 ###

**Some definitions**

Let 

$$M\in\mathbb{R}^{n\times n}$$ be an arbitrary matrix.  

Let $$p(x)=a_0 + a_1 x + a_2 x^2 + \ldots + a_n x^n \in\mathbb{R}[x]$$ be an arbitrary polynomial of less or equal to $n$.

The above polynomial can be used to define a matrix function that takes matrices as input and outputs matrices as follows: 

$$p(M) = a_0 I + a_1 M + \ldots + a_n M^n,$$ 

that is, each monomial $x^k$ is substituted by the corresponding matrix power $M^k$.

We say that a polynomial $p(x)$ annihilates a matrix $M\in\mathbb{R}^{n\times n}$ iff $p(M)=\boldsymbol{0}$, where $\boldsymbol{0}$ is the zero matrix.

**Task**

The task is to write a function ```annihilate_poly``` that takes as input an arbitrary square numpy array $M$ and outputs a vector whose cofficients are the coefficients of a (non-trivial) polynomial that annihilates $M$.  One-trivial means that its is not the zero polynomial which maps every matrix to the zero matrix.

In [0]:
import numpy as np
import numpy.linalg as NPLA
import scipy.linalg as SPLA

#references: 
#[1] https://www.answiz.com/questions/54241/python-numpy-scipy-finding-the-null-space-of-a-matrix


# vectorizes a square matrix of order n
def vec(m):
  n = np.shape(m)[0]
  m = np.array(m)
  m = np.reshape(m, (n * n, 1))
  return m

# finds m^k where m is a matrix
def mPowerK(m, k):
  return NPLA.matrix_power(m, k)

# generates the polynomial of degree k for the matrix m
def makePolynomial(m, k):
  n = np.shape(m)[0]
  polynomial = np.array([])
  for i in range(0, k + 1):
    monomial = mPowerK(m, i)
    vectorizedMonomial = vec(monomial)
    polynomial = np.append(polynomial, vectorizedMonomial)
    polynomial = np.reshape(polynomial, (n * n, i + 1), order='F')
  return polynomial

# finds the null space for the polynomial till degree k. If it does not exist,
# returns a null matrix (size 0 array)
def annihilate_poly(m, k):
  n, n = np.shape(m)
  polynomial = makePolynomial(m, k)
  nullSpace = SPLA.null_space(polynomial)
  return nullSpace

#test driver code
def main():
  n = 2
  k = 3
  m = np.array([[1, 1],
                [-1, 1]])
  m = np.reshape(m, (n, n))
  coeffs = annihilate_poly(m, k)
  print("Null Space:")
  print(coeffs)

if __name__ == "__main__":
    main()



Null Space:
[[ 0.8]
 [-0.4]
 [ 0.4]
 [ 0.2]]


**Hint**

You can reduce the problem to finding a linear dependance relationship between the $n+1$ vectors 

$$\mathrm{vec}(I), \mathrm{vec}(M), \mathrm{vec}(M^2),\ldots,\mathrm{vec}(M^n)\in\mathrm{R}^{n^2}.$$



The operation $\mathrm{vec}$ turns a square matrix $M\in\mathbb{R}^{n\times n}$ into a vector $v\in\mathbb{R}^{n^2}$ by first listing the entries of the first row, then those of the second row etc.

Update: 

To solve this problem, you have to compute the null space of the matrix $A\in \mathbb{R}^{n^2\times (n+1)}$ whose columns are the vectors $\mathrm{vec}(M^k)$ for $k\in\{0,\ldots,n\}$.


(This is not needed: 

If you don't remeber how to compute the find a linear dependance relationship, check out this stackoverflow post: https://math.stackexchange.com/questions/2198960/finding-linear-dependence-relation

You can use https://docs.scipy.org/doc/numpy/reference/generated/numpy.linalg.solve.html to solve the resulting matrix equation.)

**Task**

Write a function ```annihilate_min_deg_poly``` that computes a non-trivial polynomial that annihilates a given square matrix and has the smallest possible degree.  Recall that a polynomial $p(x)$ has degree $d$ if the coefficient $a_{d+1}=\ldots=a_n=0$.

In [0]:
import numpy as np
import numpy.linalg as NPLA
import scipy.linalg as SPLA

#references: 
#[1] https://www.answiz.com/questions/54241/python-numpy-scipy-finding-the-null-space-of-a-matrix

# vectorizes a square matrix of order n
def vec(m):
  n = np.shape(m)[0]
  m = np.array(m)
  m = np.reshape(m, (n * n, 1))
  return m

# finds m^k where m is a matrix
def mPowerK(m, k):
  return NPLA.matrix_power(m, k)

# generates the polynomial of degree k for the matrix m
def makePolynomial(m, k):
  n = np.shape(m)[0]
  polynomial = np.array([])
  for i in range(0, k + 1):
    monomial = mPowerK(m, i)
    vectorizedMonomial = vec(monomial)
    polynomial = np.append(polynomial, vectorizedMonomial)
    polynomial = np.reshape(polynomial, (n * n, i + 1), order='F')
  return polynomial

# finds the null space for the polynomial till degree k. If it does not exist,
# returns a null matrix (size 0 array)
def annihilate_poly(m, k):
  n = np.shape(m)[0]
  polynomial = makePolynomial(m, k)
  nullSpace = SPLA.null_space(polynomial, rcond=1e-10)
  return nullSpace

# finds the minimium degree polynomial 
def annihilate_min_deg_poly(m):
  n = np.shape(m)[0]
  for k in range(0, n * n + 1):
    nullSpace = annihilate_poly(m, k)
    if(nullSpace.size != 0):  # breaks on finding the 1st null space 
      break
  return nullSpace, k

# verifies test cases
def verify(m):
  n = np.shape(m)[0]
  m = np.reshape(m, (n, n))
  coeffs, k = annihilate_min_deg_poly(m)
  print("Matrix:")
  print(m)
  print("Null Space:")
  for i in range(0, coeffs.size):
    if np.allclose(coeffs[i], 0) == True:
      coeffs[i] = 0
    print(np.allclose(coeffs[i], 0))
  coeffs = np.trim_zeros(coeffs, 'b')
  print(coeffs)
  print("Min. Degree:", k)
  print()

#test driver code
def main():
  # print("diag(a, a, a) cases:")
  # print("="*20)
  # m = [[1, 0, 0], [0, 1, 0], [0, 0, 1]] 
  # verify(m)
  # m = [[2, 0, 0], [0, 2, 0], [0, 0, 2]] 
  # verify(m)
  # m = [[3, 0, 0], [0, 3, 0], [0, 0, 3]] 
  # verify(m)

  # print("diag(a, a, b) cases:")
  # print("="*20)
  # m = [[1, 0, 0], [0, 1, 0], [0, 0, 2]] 
  # verify(m)
  # m = [[2, 0, 0], [0, 2, 0], [0, 0, 1]] 
  # verify(m)
  m = [[3, 0, 0], [0, 3, 0], [0, 0, 4]] 
  # verify(m)
  # print(NPLA.matrix_power(m, 3))
  print(SPLA.null_space(m))

if __name__ == "__main__":
    main()  


[]
